## 图像处理与机器学习
### 1.安装 OpenCV-Python
OpenCV-Python就是用Python包装了C++的实现，背后实际就是C++的代码在跑，运行速度非常接近原生。

另外，图像是矩阵数据，OpenCV-Python原生支持Numpy，为矩阵运算、科学计算提供了极大的便利性。

**安装:只需终端下的一条指令：**

In [32]:
!pip install opencv-python


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [33]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


**装好OpenCV，输入import cv2，将cv2的包导入，查看OpenCV的版本，可以：**


In [34]:
import cv2

### 2.图像基本操作
**读入图像**：使用***cv2.imread()***来读入一张图片：

-参数1：图片的文件名（包含路径及后缀）

图片放在当前文件夹下，直接写文件名就行，如'cats.jpeg'
否则需要给出绝对路径，如'D:\OpenCVSamples\cats.jpeg'

-参数2：读入方式，省略即采用默认值
cv2.IMREAD_COLOR：彩色图，默认值(1)
cv2.IMREAD_GRAYSCALE：灰度图(0)
cv2.IMREAD_UNCHANGED：包含透明通道的彩色图(-1)

**小tips：路径中不能有中文，并且没有加载成功的话是不会报错的，print(img)的结果为None，后面用到时才会报错，千万别踩坑。**

In [35]:
# 读入一幅小猫图像
img = cv2.imread("cats.jpeg")

 - img 本身就是一个numpy的数组，通过`shape` `item` `size`可以查看宽高、具体元素值、图像大小

图像属性获取：img.shape获取图像的形状，图片是彩色的话，返回一个包含行数（高度）、列数（宽度）和通道数的元组，灰度图只返回行数和列数：

In [36]:
print(img.shape)
print(img.size)
# 形状中包括行数、列数和通道数
height, width, channel = img.shape
# img是灰度图的话：height, width = img.shape

(768, 1024, 3)
2359296


**显示图像**：使用***cv2.imshow()***显示图片，窗口会自适应图片的大小

-参数1：窗口的名字

-参数2：要显示的图像矩阵

In [37]:
#显示一幅小猫图像
####start code
cv2.imshow("color_cats",img)
####end code
cv2.waitKey(1000) 
cv2.destroyAllWindows()


cv2.waitKey()是让程序暂停的意思，参数是等待时间（毫秒ms）。时间一到，会继续执行接下来的程序，传入0的话表示一直等待。等待期间也可以获取用户的按键输入(Ascii码)：k = cv2.waitKey(0)。

#### 保存图像
#### 使用cv2.imwrite()保存图片，参数1是包含后缀名的文件名，参数2是图像矩阵。

In [38]:
#保存一幅小猫灰度图像
####start code

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imwrite("cat_gray.jpeg", gray)

####end code

True

是不是很简单，再接再厉哦~~~

### 3.像素点的基本操作
#### 这一小节学习获取和修改像素点的值：

通过***行列***的坐标来获取某像素点的值，对于彩色图，结果是B,G,R三个值的列表，对于灰度图或单通道图，只有一个值  

In [39]:
#获取（208，400）点的像素值
print(gray[400,208])
# 打印（208，400）点蓝色通道的值
####start code
print(img[400,208,0])

####end code

163
142


In [40]:
#将(208,400)点的像素值修改为B,G,R三通道都是255
####start code
img[400,208] = [255,255,255]
####end code

注意：行对应y，列对应x，其实是通过**img[y, x]**来访问像素点的值哦~~~
修改像素的值也是同样的方式：  

### 想一想：如何生成一个300x480 纯蓝的图？
*提示：用np.zeros([height,width,chanel],dtype='uint8')来初始化一个图像矩阵*

In [41]:
import numpy as np
z = np.zeros([480,300,3],dtype='uint8')
#### start code
z[:,:,0]= 0
z[:,:,1] = 255
z[:,:,2] = 255
cv2.imshow("black",z)
#### end code
cv2.waitKey(0) 
cv2.destroyAllWindows()

#### ROI
ROI：**Region of Interest**，感兴趣区域。比如我们要检测眼睛，眼睛肯定在脸上，所以我们可以先把脸部区域（ROI）截取出来，这样就可以大大节省计算量，提高运行速度。

截取ROI就是对图像矩阵进行切片操作。

In [42]:
img = cv2.imread("cats.jpeg")
observerArea = img[167:647,425:618]
cv2.imshow("Observer",observerArea)
cv2.waitKey(0)                            
cv2.destroyAllWindows()

> **思考** 如何将cat.jpeg的右下角水印去掉，而且尽量没有违和感

In [43]:
import cv2

img = cv2.imread('cats.jpeg')
cv2.imshow('src',img)
####start code
img[745:768,822:1024]=[255,249,250]
####end code
cv2.imshow('dst',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 4. 绘图功能
当我们找到图像中的一些特征（轮廓、边缘等），可以通过画线、圆和矩形等几何形状标记出来，也可以给图片添加文字。
绘制形状的函数有一些共同的参数，提前说明一下：
- img：要绘制形状的图片
- color：绘制的颜色：彩色图就传入BGR的一组值，如蓝色就是(255,0,0)，灰度图，传入一个灰度值就行
- thickness：线宽，默认为1；对于矩形/圆之类的封闭形状而言，传入-1表示填充形状
#### 小tips：所有绘图函数均会直接影响原图片，如果不想影响原图，可以用img.copy()生成一份拷贝再画。

#### 画线
指定起点和终点的坐标：

In [44]:
import cv2
import numpy as np

# 创建一副黑色的图像
# 画一条线宽为5的蓝色直线，参数2：起点自定义，参数3：终点自定义
####start code
img = np.zeros((512, 512,3), np.uint8)
img_line = img.copy()
cv2.line(img_line,(0,0),(512,512),(0,0,255),5)
cv2.line(img_line,(512,0),(0,512),(255,0,0),5)

####end code
cv2.imshow('img',img_line)
cv2.waitKey(0)                            
cv2.destroyAllWindows()

#### 画矩形
指定左上角和右下角的坐标：

In [45]:
# 画一个绿色边框的矩形，参数2：左上角坐标，参数3：右下角坐标
####start code
cv2.rectangle(img_line,(25,25),(175,175),(255,0,0),-1)

####end code
cv2.imshow('img',img_line)
cv2.waitKey(0)                            
cv2.destroyAllWindows()

#### 画圆
画圆需要指定圆心和半径，注意下面的例子中线宽=-1代表填充

In [46]:
# 画一个填充红色的圆，参数2：圆心坐标，参数3：半径
####start code
cv2.circle(img,(200,200),50,(255,255,0),-1)

####end code
cv2.imshow('img',img)
cv2.waitKey(0)                            
cv2.destroyAllWindows()

In [47]:
import numpy as np
import cv2
img = np.zeros((512, 512, 3), np.uint8)
img_flower = img.copy()
cv2.circle(img_flower,(200,200),50,(255,255,0),-1)
cv2.circle(img_flower,(250,250),50,(255,255,0),-1)
cv2.circle(img_flower,(200,300),50,(255,255,0),-1)
cv2.circle(img_flower,(150,250),50,(255,255,0),-1)
#cv2.circle(img_flower,(150,250),50,(255,255,0),3)
cv2.circle(img_flower,(200,250),20,(0,255,255),-1)
cv2.imshow('img',img_flower)
cv2.waitKey(0)                            
cv2.destroyAllWindows()

#### 添加文字
使用cv2.putText()添加文字，它的参数也比较多：
参数2：要添加的文本
参数3：文字的起始坐标（左下角为起点）
参数4：字体
参数5：文字大小（缩放比例）

In [48]:
# 添加文字'Intel AI Camp',位置自定义，字体为cv2.FONT_HERSHEY_SIMPLEX，大小为4，颜色自定义，线宽为2
####start code
font = cv2.FONT_HERSHEY_SIMPLEX

cv2.putText(img, 'AI for innovation Camp', (10, 500), font,
           1, (255, 255, 255), 2, lineType=cv2.LINE_AA)

####end code
cv2.imshow('img',img)
cv2.waitKey(0)                            
cv2.destroyAllWindows()

### 5.视频操作
使用摄像头，需要使用**cv2.VideoCapture(0)**创建VideoCapture对象，参数0指的是摄像头的编号，
如果你电脑上有两个摄像头的话，访问第2个摄像头就可以传入1，依此类推。
操作流程：**先占用，再读取，最后释放**


camera.read()函数返回的第1个参数ret是一个布尔值，表示当前这一帧是否获取正确。第二个参数返回当前帧的数字矩阵。

In [49]:
import cv2

###占用 0号camera
camera = cv2.VideoCapture(0)
while(camera.isOpened()):
####start code
         result, frame =   camera.read()                     # 读取摄像头采集的图像   
         if result:
             cv2.imshow("camera",frame)             # 显示该图像
    
             if cv2.waitKey(20)  == ord(' '):       # 每帧图像显示20ms，等待用户按下空格键退出
                break

#end code
camera.release()                           # 释放摄像头对象
cv2.destroyAllWindows()

#### 播放本地视频，跟打开摄像头一样，如果把摄像头的编号换成视频的路径就可以播放本地视频了。
#### 想一下cv2.waitKey()，它的参数表示暂停时间，所以这个值越大，视频播放速度越慢，反之，播放速度越快，通常设置为25或30。


In [50]:
# 播放本地视频
###读取'orange.mp4'视频文件
capture = cv2.VideoCapture('orange.mp4')
while(capture.isOpened()):
####start code
    result, frame =   capture.read()                     # 读取摄像头采集的图像   
    if result:
        cv2.imshow("camera",frame)             # 显示该图像
    
        if cv2.waitKey(20)  == ord(' '):       # 每帧图像显示20ms，等待用户按下空格键退出
            break
                              
####end code          
capture.release()
cv2.destroyAllWindows()       

### 7 颜色提取
**inRange**函数，参数有三个

第一个参数：img指的是原图

第二个参数：lower_red指的是图像中低于这个lower_red的值，图像值变为0

第三个参数：upper_red指的是图像中高于这个upper_red的值，图像值变为0

而在lower_red～upper_red之间的值变成255

mask是一个二值化的东西，也就是非黑即白


In [51]:
import cv2
import numpy as np
img = cv2.imread('opencvlogo.jpg')

####start code
lower_red = np.array ([0,200,0])                                   #设定红色logo的下限
upper_red = np.array([25,255,25])                                  #设定红色logo的上限
mask_logo = cv2.inRange(img,lower_red,upper_red)                   #取落在该范围内的所有点作为遮罩
                                                             
####end code 
cv2.imshow("mask",mask_logo)
cv2.waitKey(0)
cv2.destroyAllWindows()

可以采用inRange函数将原图进行二值化，也可以采用threshhold函数，通过设定阈值，获得二值化图像作为mask。

### 8 图像的按位操作
按位操作包括按位与/或/非/异或操作，可以实现图像的混合与局部遮挡等效果。

![jupyter](./bitwiseand.png)

掩膜(Mask)是用一副二值化图片对另外一幅图片进行局部的遮挡，“与”操作，即掩膜图像白色区域是对需要处理图像像素的保留，黑色区域是对需要处理图像像素的剔除.函数原型： 

bitwise_and(src1, src2, mask=None) 

参数说明： 
src1、src2：为输入图像, mask：图像掩膜,为8位单通道的灰度图像，输出图像像素只有mask对应位置元素不为0的部分才输出，否则该位置像素的所有通道分量都设置为0.返回值为图像矩阵。

#### 挑战练习1：提取orange.mp4视频中的橙色小球
- 读入视频中的每帧图像
- 设置颜色的上下区间，获取橙色的遮罩
- 与原图像进行按位与操作
- 显示按位与后的图像


In [52]:
import cv2   
import numpy as np

cap = cv2.VideoCapture('orange.mp4')

#参考范围：lower_orange = np.array([0,150,170])    
#参考范围：upper_orange = np.array([8,180,200])

lower_orange = np.array([50,50,160])                         #设定橙色小球的下限
upper_orange = np.array([90,90,200])                         #设定橙色小球的上限

while(cap.isOpened()):
    ret, img = cap.read()
    if ret:
      
        mask = cv2.inRange(img,lower_orange,upper_orange)                       #取落在该范围内的所有点作为遮罩
        res = cv2.bitwise_and(img,img,mask=mask)                        #用按位与运算取出橙色小球区域
        cv2.imshow('frame',img)
        cv2.imshow('mask',mask)
        cv2.imwrite("mask.jpeg",mask)  
        cv2.imshow('res',res)
      
        if cv2.waitKey(10) & 0xFF == ord(' '):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()     

In [53]:
import cv2   
import numpy as np


lower_orange = np.array([0,0,100])                         #设定橙色小球的下限
upper_orange = np.array([100,100,250])                         #设定橙色小球的上限


img = cv2.imread('ladybug.jpg')
cv2.imshow('res',img)

mask = cv2.inRange(img,lower_orange,upper_orange)                       #取落在该范围内的所有点作为遮罩
res = cv2.bitwise_and(img,img,mask=mask)                        #用按位与运算取出橙色小球区域
cv2.imshow('frame',img)
cv2.imshow('mask',mask)
cv2.imwrite("mask.jpeg",mask)  
cv2.imshow('res',res)

cv2.waitKey(0)


cv2.destroyAllWindows()     

鼠标交互：

In [54]:
img = cv2.imread('greenlight.jpg')
cv2.namedWindow('hehe')

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def onMouse(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x,y)
        hsv_value = hsv[y, x]
        print(hsv_value)
    if flags == cv2.EVENT_FLAG_LBUTTON and event == cv2.EVENT_MOUSEMOVE:
        cv2.circle(img,(x,y),3,(0,0,255),-1)
        
cv2.setMouseCallback('hehe',onMouse)

while True:
    cv2.imshow('hehe',img)
    if cv2.waitKey(10) == ord(' '):
        break
cv2.destroyAllWindows()

### 9 边缘检测与卷积运算
边缘检测是图像处理和计算机视觉中，尤其是特征提取的一个重要研究领域。边缘检测的目的是标识数字图像中亮度变化较为明显的点，这些点通常存在于目标、背景区域之间，是图像分割的重要依据。
边缘检测常用的算法有: Sobel算法、拉普拉斯算法、Canny算法等。

#### Sobel 算子

Sobel算子通过像素点空间邻域内上下、左右相邻点的灰度加权运算来模拟一阶求导,导数越大的地方说明变化越剧烈,越有可能是边缘。

Sobel 边缘检测通常带有方向性，可以只检测水平边缘或垂直边缘或都检测。
经典Sobel的卷积因子为(横向卷积因子、纵向卷积因子)：

![jupyter](./sobel.png)

用卷积的方法，通过设置不同的卷积核，对图像进行边缘检测。

In [3]:
img = cv2.imread("cats.jpeg",0)
convolutionCore = np.array((
                            [-1,0,1],
                            [-2,0,2],
                            [-1,0,1]))#拉普拉斯拓展卷积核

 
res = cv2.filter2D(img,-1,convolutionCore)                      #使用opencv的卷积函数
cv2.imshow('frame',res)

cv2.waitKey(0)                           
cv2.destroyAllWindows()

### 10 轮廓
轮廓是一系列相连的点组成的曲线，代表了物体的基本外形。
谈起轮廓不免想到边缘，它们确实很像。简单的说，轮廓是连续的，边缘并不全都连续（下图）。边缘主要是作为图像的特征使用，而轮廓主要用来分析物体的形态，比如物体的周长和面积等，可以说边缘包括轮廓。寻找轮廓的操作一般用于黑白图，所以通常会使用阈值分割或Canny边缘检测先得到二值图。

![jupyter](./contour.png)

画出mask.jpg 中最外轮廓，及其外接矩形。

![jupyter](./mask.png)


In [1]:
import cv2
img = cv2.imread("mask.png")
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)          #注意先转为灰度图才能进行阈值处理
ret,thresh = cv2.threshold(img_gray,240,255,cv2.THRESH_BINARY)
#阈值处理成二值图像

contours, hierarchy = cv2.findContours(
    thresh,
    cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(img, contours,-1, (0,0,255), 3)                  #img是彩色图像，-1 代表画出全部轮廓

x,y,w,h = cv2.boundingRect(contours[0])
img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2) 

cv2.imshow('frame',img)
cv2.waitKey(0)                           
cv2.destroyAllWindows()


#### 轮廓面积特征
通过轮廓点可以计算物体的周长、面积、质心、外接矩形、最小外接矩形等。

In [57]:
area = cv2.contourArea(contours[0]) #计算0号轮廓围起来的面积
print(area)

3630.5


>**思考** 定位orange.mp4视频中的橙色小球中心位置
- 读入视频中的每帧图像
- 设置颜色的上下区间，获取橙色的遮罩
- 获得遮罩的外层轮廓
- 根据轮廓计算面积，找到最大面积的轮廓
- 找出最大轮廓的外接矩形，并画出中心点

In [1]:
### 橙色小球跟踪实验
import cv2   
import numpy as np

cap = cv2.VideoCapture('orange.mp4')

lower_orange =  np.array([30,30,150])                                             ###请自己设定阈值
upper_orange =   np.array([80,80,220])                                                    ###请自己设定阈值

while(cap.isOpened()):
    ret, img = cap.read()
    if ret:

        area = []                                                    ##初始化面积列表
        ###start code
        mask =  cv2.inRange(img,lower_orange,upper_orange)                                                    ###请通过inRange 函数获得球的mask
        ###end code
        
        contours, hierarchy = cv2.findContours(mask,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)                                     ###通过findContours函数 找到该mask的最外层轮廓

        cv2.drawContours(img, contours,-1, (0,0,255), 3)             ###通过drawContours函数画出找到的轮廓
        
        for c in range(len(contours)):
            area.append(cv2.contourArea(contours[c]))                ####在括号中填入每个轮廓的面积
        if area:
            max_idx = np.argmax(np.array(area))
            cnt = contours[max_idx]                                  ####取出面积最大的轮廓 赋值给cnt
            
            x,y,w,h = cv2.boundingRect(cnt)                          ####用boundingRect 函数计算出该轮廓的外接矩形
            ##start code
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)   
            cv2.line(img,(int(x+w/2-10),int(y+h/2)),(int(x+w/2+10),int(y+h/2)),(255,0,0),3)
            cv2.line(img,(int(x+w/2),int(y+h/2-10)),(int(x+w/2),int(y+h/2+10)),(255,0,0),3)
            #### 画出该矩形
            ###end code
            cv2.imshow('frame',img)
       
        if cv2.waitKey(10) & 0xFF == ord(' '):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()     
#### end code

### 挑战任务:红绿灯识别
有了ROI截取、二值化、颜色区间提取、轮廓检测、掩膜、绘图等基础，我们就可以做一些很cool的识别任务啦！
有三张小车拍摄的红绿灯图像，你能从图像中识别出当前是红灯亮、绿灯亮、还是黄灯亮吗？

In [59]:
import cv2
import numpy as np

# img_all = cv2.imread("yellowlight.jpg")
img_all = cv2.imread("greenlight.jpg")
#img_all = cv2.imread("greenlight.jpg")

im_part = img_all[50:212, 708:1083]

im_part = cv2.cvtColor(im_part, cv2.COLOR_BGR2HSV)

cv2.imshow("origin",im_part)                  
cv2.waitKey(0)          

#yellow template
yellow_lower_orange =  None
yellow_upper_orange =  None

#green template 
green_lower_orange = np.array([60, 160, 160])
green_upper_orange = np.array([70, 255, 255])

#red template 
red_lower_orange =  None
red_upper_orange =  None

green_mask = cv2.inRange(im_part, green_lower_orange, green_upper_orange)
cv2.imshow("green_mask",green_mask)  

cv2.waitKey(0)     

green_contours, hierarchy = cv2.findContours(
    green_mask,
    cv2.RETR_TREE,
    cv2.CHAIN_APPROX_SIMPLE)

green_area = []

### 请找出最大面积轮廓的id号

for c in range(len(green_contours)):
    green_area.append(cv2.contourArea(green_contours[c]))
if green_area:
    green_max_idx = np.argmax(np.array(green_area))

if green_area:
    #####请用boundingRect 函数计算出该轮廓的外接矩形
    #### 请用绿色画出该矩形
    ###start code
    green_cnt = green_contours[green_max_idx]
    x,y,w,h = cv2.boundingRect(green_cnt)
    im_part=cv2.rectangle(im_part,(x,y),(x+w,y+h),(0,255,255),2)
    im_part=cv2.line(im_part,(int(x+w/2),int(y+h/2-2)),(int(x+w/2),int(y+h/2+2)),(255,0,0),2)
    im_part=cv2.line(im_part,(int(x+w/2-2),int(y+h/2)),(int(x+w/2+2),int(y+h/2)),(255,0,0),2)
    
    ###end code    
    print("only green light on!")
else:
    print("no light on!")

                                            ###将处理好的部分图像赋值给原图
cv2.imshow("contour",img_all) 
cv2.waitKey(0)                            
cv2.destroyAllWindows()

only green light on!


### Bonus ！！补充内容：12 OpenCV 中的机器学习模型
OpenCV中有很多机器学习模块，比如人脸检测、SVM分类器。以人脸检测为例：OpenCV 提供了两种人脸检测方法，包括：
- 基于 Haar 级联的人脸检测器
- 基于深度学习的人脸检测器
基于 Haar 级联的人脸检测器,OpenCV 提供了 4 个级联分类器用于人脸检测，可以从OpenCV 官方下载这些级联分类器文件：
- haarcascade_frontalface_alt.xml 
- haarcascade_frontalface_alt2.xml 
- haarcascade_frontalface_alt_tree.xml 
- haarcascade_frontalface_default.xml 
总的来说这些分类器具有相似的准确率.

OpenCV 提供了 cv2.CascadeClassifier() 函数用于从文件中加载分类器：

In [60]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

接下来，就可以使用 cv2.CascadeClassifier.detectMultiScale() 函数执行检测了：

In [61]:
faces_rects = faceCascade.detectMultiScale(img,scaleFactor = 1.05,minNeighbors = 5,minSize = (30,30),flags = cv2.CASCADE_SCALE_IMAGE)

cv2.CascadeClassifier.detectMultiScale() 函数检测对象并将它们作为矩形列表返回。下面是完整的视频流人脸检测代码：

In [62]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, img = cap.read()
    print(ret)
    if ret:  
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces_rects = faceCascade.detectMultiScale(img,scaleFactor = 1.05,minNeighbors = 5,minSize = (30,30),flags = cv2.CASCADE_SCALE_IMAGE)
        
        for (x,y,w,h) in faces_rects:
            img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
           
            cv2.imshow('Press Spacebar to Exit',img) 

        if cv2.waitKey(10) & 0xFF == ord(' '):  # 按下空格键退出
            break
    else:
        break

cap.release()                          
cv2.destroyAllWindows()

True
True


OpenCV自带了人脸分类器、人眼检测器、姿态检测、表情识别等分类器。在目录anaconda3\Lib\site-packages\cv2\data下的xml文件中，用到的时候记得把他们copy到自己的folder下。
OpenCV也预训练好了人眼检测模型，haarcascade_eye.xml.可通过 eyes = eye_cascade.detectMultiScale(img)得到眼睛区域。